IMPORTAR LIBRERIAS

In [1]:
import pandas as pd 
import json
from pandas import json_normalize
import ast
import re
import numpy as np

C:\Users\argen\AppData\Local\Temp\ipykernel_42980\2029532637.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


CARGA DE DATOS

In [2]:
df = pd.read_csv("./Movies/movies_dataset.csv")
df_2 = pd.read_csv("./Movies/credits.csv")

C:\Users\argen\AppData\Local\Temp\ipykernel_42980\1457517589.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Movies/movies_dataset.csv")


DESANIDAR COLUMNAS

FUNCIONES

In [3]:
def carga_json_loads(x):
    if isinstance(x, str):
        try:
            return json.loads(x)
        except json.JSONDecodeError:
            return None  # O puedes devolver un valor predeterminado o manejarlo de otra manera
    return x


def fix_incomplete_strings(entry):
    if isinstance(entry, str):
        try:
            return ast.literal_eval(entry)  # Intentar evaluar directamente
        except (ValueError, SyntaxError):  # Si falla, limpiar la cadena
            # Reemplazar comillas simples por dobles para estandarizar el formato
            cleaned_entry = entry.replace("'", "\"")
            try:
                return ast.literal_eval(cleaned_entry)  # Intentar nuevamente después de limpiar
            except (ValueError, SyntaxError):
                return None  # Si todavía falla, devolver None para revisión
    return entry  # Devolver los valores no string (como NaN) tal cual

DESANIDADO DE COLUMNAS DEL DATASETS MOVIES

In [4]:
df["belongs_to_collection"] = df["belongs_to_collection"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df["belongs_to_collection"] = df["belongs_to_collection"].apply(carga_json_loads)
collection = json_normalize(df["belongs_to_collection"])
collection = collection.rename(columns= {'id': 'collection_id', 'name':'collection_name'})
df = pd.concat([df, collection['collection_id'], collection['collection_name']], axis=1)
df.drop(columns='belongs_to_collection', inplace= True)

In [5]:
df["genres"] = df["genres"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['genres'] = df['genres'].apply(ast.literal_eval)
data_exploded = df.explode('genres')
data_normalized = json_normalize(data_exploded['genres']).rename(columns= {'id': 'genres_id', 'name':'genres_name'})
df = pd.concat([data_exploded.reset_index(drop=True), data_normalized], axis=1)
df.drop(columns='genres', inplace= True)

In [6]:
df["production_companies"] = df["production_companies"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['production_companies'] = df['production_companies'].apply(fix_incomplete_strings)
data_exploded = df.explode("production_companies")
data_normalized = json_normalize(data_exploded["production_companies"]).rename(columns= {'id': 'production_companies_id', 'name':'production_companies_name'})
df = pd.concat([data_exploded.reset_index(drop=True), data_normalized], axis=1)
df.drop(columns="production_companies", inplace= True)


In [7]:
df["production_countries"] = df["production_countries"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['production_countries'] = df['production_countries'].apply(fix_incomplete_strings)
data_exploded = df.explode('production_countries')
data_normalized = json_normalize(data_exploded['production_countries']).rename(columns= {'id': 'production_countries_id', 'name':'production_countries_name'})
df = pd.concat([data_exploded.reset_index(drop=True), data_normalized], axis=1)
df.drop(columns="production_countries", inplace= True)

In [8]:
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['spoken_languages'] = df['spoken_languages'].apply(fix_incomplete_strings)
data_exploded = df.explode('spoken_languages')
data_normalized = json_normalize(data_exploded['spoken_languages']).rename(columns= {'id': 'spoken_languages_id', 'name':'spoken_languages_name'})
df = pd.concat([data_exploded.reset_index(drop=True), data_normalized], axis=1)
df.drop(columns="spoken_languages", inplace= True)

TRANSFORMACIONES

Valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [9]:
#Verificar si tiene nuloss y cuantos
print(df.revenue.isnull().sum())
print(df.budget.isnull().sum())

21
0


In [10]:
#rellenar los nulos con cero de la columna 'revenue'
df.loc[df.revenue.isnull()] = 0

Los valores nulos del campo release date deben eliminarse.

In [12]:
#verificar que la colunma 'release_date' tenga nulos y ceros
print(df['release_date'].isnull().sum())
print(df['release_date'].value_counts()[0])

170
21


In [13]:
s = df[df['release_date'] == 0]

In [14]:
s.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,collection_id,collection_name,genres_id,genres_name,production_companies_name,production_companies_id,iso_3166_1,production_countries_name,iso_639_1,spoken_languages_name
229629,0,0,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0.0,0,0,0,0
229630,0,0,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0.0,0,0,0,0
229631,0,0,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0.0,0,0,0,0
229632,0,0,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0.0,0,0,0,0
229633,0,0,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0,0,0.0,0,0,0,0


In [15]:
#Eliminar filas sin datos
df = df[(df != 0).any(axis=1)]
#Eliminar filas valores nulos de columna 'release_date'
df.dropna(subset=['release_date'], inplace=True)

Camnio de Formato en las columnas con fecha ('release_date')

In [16]:
#Cambiar el tipo de dato de la columna 'release_date'
df['release_date'] =pd.to_datetime(df['release_date'])
#Crear la columna 'release_year'
df['release_year']= df['release_date'].dt.year

Creacion de la columna con el retorno de inversión ('return'), con los campos revenue y budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [17]:
#Cambio del tipo de dato de la columna 'budget'
df['budget'] = df['budget'].astype(float)
#Creacion de la columna Retorno de Inversion ('return')
df['return'] = ((df['revenue'] / df['budget'].replace(0, np.nan)).round(2)).fillna(0)

Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [18]:
df.drop(columns= ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [19]:
#Cambio de tupo de datos de otras columnas
df = df.astype({'id': int, 'popularity': float})

DESANIDADO DE COLUMNAS DEL DATASETS CREDITS

In [20]:
#Creacion de los DataFrames cast y crew
df_cast = df_2[['cast', 'id']]
df_crew = df_2[['crew', 'id']]

In [21]:
# Desanidado del Dataframe 'cast'
df_cast["cast"] = df_cast["cast"].apply(fix_incomplete_strings)
df_cast["cast"] = df_cast["cast"].apply(carga_json_loads)
data_exploded = df_cast.explode("cast")
data_normalized = json_normalize(data_exploded["cast"]).rename(columns= {'credit_id': 'credit_id_cast', 'gender':'cast_gender','id': 'actor_id', 'name':'actor_name', 'profile_path':'cast_profile_path'})
df_cast = pd.concat([data_exploded.reset_index(drop=True), data_normalized], axis=1)
df_cast.drop(columns= ['cast','credit_id_cast','cast_gender', 'order','cast_profile_path' ], inplace=True)

C:\Users\argen\AppData\Local\Temp\ipykernel_42980\4112066881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cast["cast"] = df_cast["cast"].apply(fix_incomplete_strings)
C:\Users\argen\AppData\Local\Temp\ipykernel_42980\4112066881.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cast["cast"] = df_cast["cast"].apply(carga_json_loads)


In [22]:
#Desanidado del DataFrame 'crew'
df_crew["crew"] = df_crew["crew"].apply(fix_incomplete_strings)
df_crew["crew"] = df_crew["crew"].apply(carga_json_loads)
data_exploded = df_crew.explode("crew")
data_normalized = json_normalize(data_exploded["crew"]).rename(columns= {'credit_id': 'credit_id_crew', 'gender':'crew_gender','id': 'crew_id', 'name':'crew_name', 'profile_path':'crew_profile_path'})
df_crew = pd.concat([data_exploded.reset_index(drop=True), data_normalized], axis=1)
df_crew.drop(columns=["crew", "credit_id_crew", "department", "crew_gender", "crew_profile_path"], inplace= True)


C:\Users\argen\AppData\Local\Temp\ipykernel_42980\396785367.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crew["crew"] = df_crew["crew"].apply(fix_incomplete_strings)
C:\Users\argen\AppData\Local\Temp\ipykernel_42980\396785367.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crew["crew"] = df_crew["crew"].apply(carga_json_loads)


GENERAR DATASETS PARA CONSULTAS

In [24]:
df_crew.to_csv('crew.csv', index=False)
df_cast.to_csv('cast.csv', index=False)
datasets = df[['id','popularity', 'release_date', 'status', 'title', 'vote_average', 'vote_count','return','budget','revenue']]
datasets.drop_duplicates(inplace=True)
datasets.to_csv('dataset.csv', index=False)

C:\Users\argen\AppData\Local\Temp\ipykernel_42980\217629843.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets.drop_duplicates(inplace=True)
